In [1]:
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers.convolutional import Conv3D, MaxPooling3D
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import GlobalMaxPool2D
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import Reshape, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.python.keras.utils import np_utils, generic_utils

import os
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import cv2
from sklearn import preprocessing

In [25]:
nb_classes = 2
dropout_rate = 0.2

model = Sequential()

strides = (1,1,1)
kernel_size = (3, 3, 3)

model.add(Conv3D(32, kernel_size, strides=strides, activation='relu', padding='same', input_shape=(30, 96, 144, 3)))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)
#model.add(Dropout(dropout_rate))

model.add(Conv3D(64, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)
#model.add(Dropout(dropout_rate))

model.add(Conv3D(128, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)
#model.add(Dropout(dropout_rate))

model.add(Conv3D(256, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
#model.add(Dropout(dropout_rate))

model.add(Conv3D(256, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
#model.add(Dropout(dropout_rate))

model.add(MaxPooling3D(pool_size=(1,12,18)))
print(model.output_shape)

model.add(Reshape((30, 256)))
print(model.output_shape)
model.add(LSTM(256, return_sequences=True))
print(model.output_shape)
model.add(LSTM(256))
print(model.output_shape)

model.add(Dense(256, activation='relu'))
print(model.output_shape)
#model.add(Dropout(dropout_rate))

model.add(Dense(nb_classes, activation='sigmoid'))
print(model.output_shape)

(None, 30, 96, 144, 32)
(None, 30, 48, 72, 32)
(None, 30, 48, 72, 64)
(None, 30, 24, 36, 64)
(None, 30, 24, 36, 128)
(None, 30, 12, 18, 128)
(None, 30, 12, 18, 256)
(None, 30, 12, 18, 256)
(None, 30, 1, 1, 256)
(None, 30, 256)
(None, 30, 256)
(None, 256)
(None, 256)
(None, 2)


In [26]:
def show(img, title="img"):
    cv2.imshow(title, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [27]:
# Resize frames

def resize_frame(frame, size):
    frame = img.imread(frame)
    frame = cv2.resize(frame, size)
    return frame

In [28]:
from video_datagen import VideoDataGenerator

gen = VideoDataGenerator()

In [29]:
sgd = SGD(lr=0.0001)
model.compile(optimizer=sgd, loss= 'binary_crossentropy', metrics=['accuracy'])

In [30]:
training_path = '/home/siddhant/Datasets/Crash Dataset/Final_Preprocessed/train'
validation_path = '/home/siddhant/Datasets/Crash Dataset/Final_Preprocessed/validation'

batchsize = 2
training_set = gen.flow_from_directory(training_path, target_size=(96, 144), batch_size=batchsize, clip_size=30)
validation_set = gen.flow_from_directory(validation_path, target_size=(96, 144), batch_size=8, clip_size=30)

Found 2564 videos belonging to 2 classes.
Found 225 videos belonging to 2 classes.


In [18]:
weights = {0:1.0, 1:3}

In [19]:
training_history = model.fit_generator(training_set, use_multiprocessing=True,
                                       epochs =20,validation_data = validation_set , verbose = 1 , validation_steps = 1,
                                       steps_per_epoch=2564//2)

Epoch 1/20
1282/1282 [==============================] - 998s 778ms/step - loss: 0.6227 - accuracy: 0.6899 - val_loss: 0.5634 - val_accuracy: 0.6875
Epoch 2/20
1282/1282 [==============================] - 1003s 782ms/step - loss: 0.5303 - accuracy: 0.8309 - val_loss: 0.4947 - val_accuracy: 0.8750
Epoch 3/20
1282/1282 [==============================] - 1003s 782ms/step - loss: 0.4522 - accuracy: 0.8752 - val_loss: 0.6947 - val_accuracy: 0.5000
Epoch 4/20
1282/1282 [==============================] - 1000s 780ms/step - loss: 0.3726 - accuracy: 0.9119 - val_loss: 0.2808 - val_accuracy: 1.0000
Epoch 5/20
1282/1282 [==============================] - 1001s 781ms/step - loss: 0.2899 - accuracy: 0.9405 - val_loss: 0.1737 - val_accuracy: 1.0000
Epoch 6/20
1282/1282 [==============================] - 992s 774ms/step - loss: 0.2412 - accuracy: 0.9524 - val_loss: 0.2633 - val_accuracy: 0.8750
Epoch 7/20
1282/1282 [==============================] - 992s 774ms/step - loss: 0.2032 - accuracy: 0.9563 - 

In [20]:
model.save_weights('model-sgd-0001-20epochs2.h5')

In [7]:
model.load_weights('model-sgd-0001-20epochs.h5')

In [32]:
testing_path = '/home/siddhant/Datasets/Crash Dataset/Final_Preprocessed/test'

testing_set = gen.flow_from_directory(testing_path, target_size=(96, 144), batch_size=10, clip_size=30)

Found 208 videos belonging to 2 classes.


In [36]:
inst = next(testing_set)

In [37]:
score = model.evaluate(inst[0], inst[1], verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.6991062760353088
Test accuracy: 0.4


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 96, 144, 32)   2624      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 96, 144, 32)   128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 30, 48, 72, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 30, 48, 72, 64)    55360     
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 48, 72, 64)    256       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 30, 24, 36, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 24, 36, 128)   2

In [10]:
import pickle

In [38]:
to_predict = []
classes = ['Crash', 'NotCrash']

model.load_weights('model-sgd-0001-20epochs.h5')
num_frames = 0
cap = cv2.VideoCapture('/home/siddhant/Datasets/Crash Dataset/crashCompilationTesting.mp4')

#cap.set(12, 50)
#cap.set(6, 10)
#cap.set(cv2.CAP_PROP_FPS, 10)

preds = []

classe = ''
import time
fps = 30
counter  = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame_cp = frame
    
    frame_cp = cv2.resize(frame, (144, 96))
    
    if counter%5 == 0:
        to_predict.append(frame_cp)
    counter += 1
    predict = 0
    if len(to_predict) == 30:

        frame_to_predict = [[]]
        frame_to_predict[0] = np.array(to_predict, dtype=np.float32)


        predict = model.predict(np.array(frame_to_predict))
        classe = classes[np.argmax(predict)]
        if np.amax(predict) > 0.60:
            print('Class = ',classe, 'Precision = ', np.amax(predict)*100,'%')
            preds.append(np.argmax(predict))
           # with open('gesture.pkl','wb') as f:
                #pickle.dump(np.argmax(predict), f)
        if len(preds) >= 10:
            preds = preds[8:9]

        to_predict = []
        time.sleep(0.1)
        font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, classe, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0),1,cv2.LINE_AA)
    
    cv2.imshow('Live Accident Detection',frame)
    if cv2.waitKey(int( (1 / int(fps)) * 1000)) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Class =  Crash Precision =  93.42271685600281 %
Class =  Crash Precision =  91.50057435035706 %
Class =  Crash Precision =  93.78187656402588 %


In [11]:
def readSamples(pathToSample, resize_dim = (144, 96)):
    # pathToSample points to dir containing the frames
    frames = os.listdir(pathToSample)
    frames.sort()
    sample = []
    for frame in frames:
        im = cv2.imread(f'{pathToSample}/{frame}')
        im = cv2.resize(im, resize_dim)
        sample.append(im)
    return sample

In [12]:
sample = readSamples('/home/siddhant/Datasets/Crash Dataset/Final_Preprocessed/test/Crash/22')
sample = [sample]
sample = np.array(sample, dtype=np.float32)
model.predict(sample)

array([[0.9771811 , 0.02018946]], dtype=float32)

In [35]:
np.shape(sample)

(1, 30, 96, 144, 3)